In [18]:
!pip install torch==1.8.1 torchvision==0.9.1 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached torch-1.8.1-cp39-none-macosx_10_9_x86_64.whl (119.6 MB)
     |████████████████████████████████| 13.1 MB 374 kB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 2.4 MB/s eta 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1


In [19]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [5]:
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [6]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [7]:
result = model(tokens)

In [8]:
result.logits

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward>)

In [9]:
int(torch.argmax(result.logits))+1

4

In [22]:
r = requests.get('https://www.yelp.com/biz/mariel-boston-3')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [23]:
reviews

["The ambiance is amazing, but the food was underwhelming. we ordered 2 apps and 2 small plates to share, and I wouldn't reorder anything if I went back. also, our food came out within 3 minutes of ordering it-that was awkward and left us feeling like it wasn't made to order. our waiter was lovely, though!",
 'Great time at Mariel- wonderful service, gorgeous interior with romantic ambiance, and the food/drinks exceeded expectations. In particular, we loved the fufu gnocchi and the mojito. Feels like a special place to celebrate an occasion. We ordered:+ Yuca Cheese Puffs+ Chicken Tots+ Steak Churrasco + Fufu GnocchiDrinks+Lulo Daiquiri+Old Habana (Guava) Mojito',
 "You come here for the ambiance. The food here is good and they're served tapas styled, but a bit overpriced. The ambiance though is top tier. Its trendy, artsy, dimly lit and my friend says it looks like Cuba inside. I've been here a couple of times. The last time was on a Saturday night and we hung out by the bar. The DJ w

In [24]:
import numpy as np
import pandas as pd

In [25]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [26]:
df['review'].iloc[0]

"The ambiance is amazing, but the food was underwhelming. we ordered 2 apps and 2 small plates to share, and I wouldn't reorder anything if I went back. also, our food came out within 3 minutes of ordering it-that was awkward and left us feeling like it wasn't made to order. our waiter was lovely, though!"

In [27]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [28]:
sentiment_score(df['review'].iloc[1])

5

In [29]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [30]:
df

,review,sentiment
0,"The ambiance is amazing, but the food was unde...",3
1,"Great time at Mariel- wonderful service, gorge...",5
2,You come here for the ambiance. The food here ...,3
3,I have been wanting to come to Mariels tor the...,4
4,Visited on a Saturday night in celebration of ...,5
5,Cannot recommend this place enough! I have bee...,2
6,The ambiance here is amazing. Honestly paying ...,4
7,I had such a great time here at Mariels. We ca...,5
8,Mariel is pricey but one of my favorite food p...,5
9,"Service, experience, tapas, unwired drinks, an...",4


In [31]:
df['review'].iloc[3]

"I have been wanting to come to Mariels tor the longest time and finally came here and last night. The restaurant is very beautifully decorated and the ambiance is amazing. The music isn't too loud and the service is top notch. Our waitress was very attentive that night and food was brought out very fast. My only complaint was that my calamari was very very very salty. I always get calamari at other places because you can't go wrong with calamari but somehow they managed to make it taste very bad, which was disappointing. I did enjoy the Yuca cheese puffs and my Pina Colada drink so I will give them that. I will definitely come back in the future to try the rest of the other food items on the menu, I loved the service. Hopefully next time the food will be slightly better."

In [32]:
df['review'].iloc[4]

"Visited on a Saturday night in celebration of our wedding anniversary. We've been trying to make it Mariel for the longest time. We arrived 10 minutes early and our table wasn't quite ready. We grabbed a drink at the bar while we awaited seating. The bar and surrounding was jam packed but the bar tenders were very attentive and handed us a menu. We orders and old fashioned and a glass of wine. We marveled the architecture and the decor while sipping our drinks. We were seated promptly at 6:30 in the dining room. Sarah, our waitress, was wonderful, she saw uh oooh-ing and aw-ing over the daiquiri tree and allowed us to order one even though there was only two of us! The drinks and food were fantastic. Unfortunately we didn't save any room for dessert. All in all, it was a wonderfulExperience and we will most definitely be back."

In [34]:
r = requests.get('https://www.yelp.com/biz/ruka-restobar-boston')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [35]:
reviews

['In town for business, had dinner here twice with a small group each time.Stayed at the Godfrey Busy on weeknights - incredible food, service, and will be back.',
 'This place lives up to its raving reviews. If you plan to dine alone, you are lucky to get a seat at the bar. If you successfully do get a chair, I hope you consider ordering two items. I chose a serving of a dish with cooked trumpet mushrooms and crispy tofu to start. Then, I went for a sushi roll called Phoenix Sunrise, a roll with rice on the outside and topped with torched salmon. Both were delicious, but this roll was the star of the show. I also enjoyed the sake they had, which paired well with both dishes. The wait staff and the bar staff were nice and helpful. If you are dining alone, I would bring your work computer with you because it is a very busy and noisy restaurant. The food is great, but it can be difficult to hear yourself think. Also, this place is on the expensive side. However, if you are in Boston for 

In [36]:
import numpy as np
import pandas as pd

In [37]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [38]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [39]:
sentiment_score(df['review'].iloc[1])

4

In [40]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [41]:
df

,review,sentiment
0,"In town for business, had dinner here twice wi...",5
1,This place lives up to its raving reviews. If ...,4
2,All hits no misses is how I'd describe the foo...,5
3,I came here for a late night bite with a frien...,2
4,"Still 5 stars!The atmosphere is so fun, and go...",5
5,It was so hard to decide what to get! Everythi...,2
6,Nothing but good things for this place! The am...,5
7,"The server Chris was fantastic, he was extreme...",4
8,Great dinner last night in Boston with family....,5
9,Food was overall very average. The korean stea...,3


In [42]:
df['review'].iloc[3]

'I came here for a late night bite with a friend and was thoroughly impressed! We got a bottle of sake, which came in a fun ice tub filled with dry ice.We then shared the Tuna Katari, the king Tupac, the Lima Style Fried Calamari, and pork belly buns. The restaurant had great ambiance, and we really enjoyed all of the food that we ate! I definitely will return and look forward to trying more of the dishes on the menu!'

In [50]:
score = 0
for i in range(11):
    score += sentiment_score(df['review'].iloc[i])
score/10

4.6